In [1]:
import astroprov

from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs

from astroquery.simbad import Simbad

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from photutils import DAOStarFinder
from photutils import CircularAperture
import os
import subprocess

import image_registration
from image_registration import chi2_shift
from image_registration.fft_tools import shift
import pandas
import collections

from reproject import reproject_interp
from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS

In [2]:
imageDirectory = '/data/mj1e16/kepler/cal/'
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0] != '.']
#dirlist = sort(dirlist)
imagelist = [imageDirectory+x for x in dirlist]
#print(dirlist)

In [3]:
def getSkyGroup(imagelist,image,channel):
    hdu_list = fits.open(imagelist[image])
    hdr = hdu_list[channel].header
    location = hdr['SKYGROUP']
    return location

In [4]:
def getSkyGroup(image,channel):
    hdu_list = fits.open(image)
    hdr = hdu_list[channel].header
    location = hdr['SKYGROUP']
    return location

In [5]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [6]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [7]:
def medianAlign(im1_aligned_to_im2,hdulist,hduchan,hdunames):
    shaperoni = im1_aligned_to_im2.shape
    xsegments = int(shaperoni[0]/100)
    ylen = shaperoni[0] 

    alignedHDU = [hdulist[0]]
    im2 = hdulist[0]
    for image in range(1,len(hdulist)):
        im1 = hdulist[image]
        yoff,xoff = image_registration.cross_correlation_shifts(im1,im2)
        im1_aligned_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)
        alignedHDU.append(im1_aligned_to_im2)
        
        print(getSkyGroup(hdunames[image],hduchan[image]))
        print(hdunames[image],hduchan[image])
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(im1_aligned_to_im2,cmap='Greys',norm=LogNorm())
        plt.title('Median Image Align ')
        plt.show()


    output = np.zeros(shaperoni)
    for i in range(100):
        imageSegment = []
        j = i * xsegments
        k = j + xsegments
        for imData in range(len(alignedHDU)):
            imageSegment.append(alignedHDU[imData][j:k,:])

        output[j:k,:] = np.median(imageSegment,axis=0)
    if k != shaperoni[1]: # is this the correct way round?
        imageSegment = []
        print(k,shaperoni[1],'k - shaperoni')
        for imData in range(len(alignedHDU)):
            imageSegment.append(alignedHDU[imData][k:shaperoni[0],:])
        output[k:shaperoni[0],:] = np.median(imageSegment,axis=0)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(output,cmap='Greys',norm=LogNorm())
    plt.title('Median Image Align ')
    plt.show()
    
    return output


In [8]:
def findTemplateImage(imagelist,channelList,channel,ximage=1070.,yimage=1140.):
    image = 0
    averageFlux = 0

    for refimage in range(len(imagelist)):
        chanNoList = int(channelList[refimage][channel])
        chanNoImName = imagelist[refimage]
        im2 = getImageData(chanNoImName,chanNoList)
        averageFlux = int((sum(sum(im2)))/(ximage*yimage))
        if averageFlux > 1:
            break
        
    goodImage = {'chanNo':chanNoList,'chanNoImName':chanNoImName}
    return goodImage

In [9]:
def findSpecificTemplateImage(imagelist,channelList,channel,specificChannel,ximage=1070.,yimage=1140.):
    image = 0
    averageFlux = 0

    for refimage in range(len(imagelist)):
        chanNoList = int(channelList[refimage][channel])
        if chanNoList == specificChannel:
            chanNoImName = imagelist[refimage]
            im2 = getImageData(chanNoImName,chanNoList)
            averageFlux = int((sum(sum(im2)))/(ximage*yimage))
        if averageFlux > 1:
            break
        
    goodImage = {'chanNo':chanNoList,'chanNoImName':chanNoImName}
    return goodImage

In [10]:
def newMedian(imageList,channelList,outfile='blah',ximage=1070,yimage=1140):
    
    image_concat = []
    medianlist = []

    ccds = [44,63,79]
    
    #for channel in range(len(channelList)):
    for channel in ccds:
        
        hdulist = []
        goodImage = findTemplateImage(imagelist,channelList,channel)
        chanNoList = [goodImage['chanNo']]
        chanNoImName = [goodImage['chanNoImName']]
        im2 = getImageData(chanNoImName[0],chanNoList[0])
        
        for image in range(1,len(imagelist)):
            chanNo = int(channelList[image][channel])
            im1 = getImageData(imagelist[image],chanNo)            
            averageFlux = int((sum(sum(im1)))/(ximage*yimage))
            print(averageFlux)
            if averageFlux > 1:
                
#                 yoff,xoff = image_registration.cross_correlation_shifts(im1,im2)
#                 im1_aligned_to_im2 = np.roll(np.roll(im1,int(yoff),1),int(xoff),0)
                hdulist.append(im1)
                
            
        output = medianAlign(im1,hdulist)
        print(chanNoList)
        print(chanNoImName)
        hdu = fits.PrimaryHDU(output)
        hdu.writeto(outfile+'_{}.fits'.format(channel), clobber=True)
        medianlist.append(outfile+'_{}.fits'.format(channel))
        
        
    return medianlist
    
    

In [16]:
def selectiveMedian(imageList,channelList,outfile='blah',ximage=1070,yimage=1140):
    
    image_concat = []
    medianlist = []

    ccds = [63,79]
    #for channel in range(10,11): #len(channelList[0])):
    for channel in ccds:
        hdulist = [[],[],[],[]]
        hduchans = [[],[],[],[]]
        hdunames = [[],[],[],[]]
        offSet =  1000
        refimage = 0
        image = 0
        
        goodImage = findTemplateImage(imagelist,channelList,channel)
        chanNoList = [goodImage['chanNo']]
        chanNoImName = [goodImage['chanNoImName']]
        hdulist[0].append(getImageData(chanNoImName[0],chanNoList[0]))
        hduchans[0].append(chanNoList[0])
        hdunames[0].append(chanNoImName[0])
        #print(getSkyGroup(imagelist,)
        print(hduchans)
        for image in range(1,12): #len(imagelist)):
            chanNo = int(channelList[image][channel])
            #print(getSkyGroup(imagelist,image,chanNo),'skygroup ####')
            if chanNo in chanNoList: 
                loc = chanNoList.index(chanNo)
                print(chanNoList[loc],chanNo)
                im1 = getImageData(imagelist[image],chanNo)
                averageFlux = int((sum(sum(im1)))/(ximage*yimage))
                #print(averageFlux)
                if averageFlux > 1:
                    

                    if chanNo == chanNoList[loc]:
                        hdulist[loc].append(im1)
                        print(chanNo)
                        hduchans[loc].append(chanNo)
                        print(hduchans)
                        hdunames[loc].append(imagelist[image])

                    
     
                
            else:
                goodImage = findSpecificTemplateImage(imagelist,channelList,channel,chanNo)
                chanNoList.append(goodImage['chanNo'])
                chanNoImName.append(goodImage['chanNoImName'])
                hdulist[len(chanNoList)-1].append(getImageData(chanNoImName[-1],chanNoList[-1]))
                
                hduchans[len(chanNoList)-1].append(chanNoList[-1])
                hdunames[len(chanNoList)-1].append(chanNoImName[-1])
                print(hduchans,'starter')
                print(chanNoList[-1],'new')

        print(hduchans)
        for loc in range(len(hdulist)):
            output = medianAlign(im1,hdulist[loc],hduchans[loc],hdunames[loc])        
            print(chanNoList)
            print(chanNoImName)
            hdu = fits.PrimaryHDU(output)
            hdu.writeto(outfile+'_{}_{}.fits'.format(channel,chanNoList[loc]), clobber=True)
            medianlist.append(outfile+'_{}_{}.fits'.format(channel,chanNoList[loc]))
        
        
    return medianlist
    
    

In [12]:
extensionOrders = []
for image in imagelist:
    extensionOrders.append(correctExtensionOrder(image))

In [13]:
medianDir = '/data/mj1e16/kepler/newDiff/'
medians = os.listdir(medianDir)
medians = [x for x in medians if x[0] != '.']
medians.sort()

In [18]:
#selectiveMedian(imagelist,extensionOrders,outfile='/data/mj1e16/kepler/medians/meidian')

In [19]:
#newMedian(imagelist,extensionOrders,outfile='/data/mj1e16/kepler/medians/meidian')

In [ ]:
[[41, 42, 43, 41, 41, 43, 42, 41, 44, 42, 42, 41], [41, 42, 43, 41, 41, 43, 42, 41, 44, 42, 42, 41], [41, 42, 43, 41, 41, 43, 42, 41, 44, 42, 42, 41], [41, 42, 43, 41, 41, 43, 42, 41, 44, 42, 42, 41]]